# Размещение баннеров

## Постановка задачи
Представим, что международное круизное агентство "Carnival Cruise Line" решило себя разрекламировать с помощью баннеров и обратилось для этого к Вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с Вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры - легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях - Foursqare.
Часть открытых данных есть, например, на сайте archive.org:
https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив с этой страницы.

## Преобразование данных
Нас будет интересовать файл checkins.dat. 
Для удобной работы с этим документом преобразуем его к формату csv, удалив строки не содержащие координат - они неинформативны для нас:

id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...

С помощью pandas построим DataFrame и убедимся, что все 396632 строк с координатами считаны успешно.

In [ ]:
import pandas as pd

In [ ]:
lines = []
data = []

df = pd.read_csv('checkins.csv')

In [ ]:
df = df[(df.latitude != '') & (df.longitude != '')]
df.head()

In [ ]:
df.shape

# Кластеризация
Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача - хороший повод познакомиться с алгоритмом MeanShift. Его описание можно посмотреть в [sklearn user guide](http://scikit-learn.org/stable/modules/clustering.html#mean-shift). Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках, кластеризация будет работать долго (порядка часа). Для получения корректного ответа достаточно и 100000 (~2 минуты на "среднем" ноутбуке).

In [ ]:
from sklearn.cluster import MeanShift

In [ ]:
data = df[['latitude', 'longitude']]
data = data.applymap(float)

In [ ]:
data_sample = data.sample(100000)

In [ ]:
%%time
# Применяем MeanShift
# обучаем модель (fit) на data_sample

Некоторые из получившихся кластеров содержат слишком мало точек - такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [ ]:
data_sample['cluster'] = clst.predict(data_sample)

In [ ]:
cluster_size = pd.DataFrame(data_sample.pivot_table(index = 'cluster', aggfunc = 'count', values = 'latitude'))
cluster_size.columns = ['clust_size']

In [ ]:
cluster_centers_df = pd.DataFrame(clst.cluster_centers_)
cluster_centers_df.columns = ['cent_latitude', 'cent_longitude']

In [ ]:
cluster_df = cluster_centers_df.join(cluster_size)
cluster_df.to_csv('clusters.csv', index = None)
# здесь пишем код, который оставляет в таблице cluster_df только те кластеры, размер которых (clust_size) больше 15
cluster_df.head()

Чтобы увидеть получившиеся результаты на карте, откройте сгенерированный файл clusters.csv и передайте центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071

...

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу "Carnival Cruise Line" адреса офисов:

 * 33.751277, -118.188740 (_Los Angeles_)
 * 25.867736, -80.324116 (_Miami_)
 * 51.503016, -0.075479 (_London_)
 * 52.378894, 4.885084 (_Amsterdam_)
 * 39.366487, 117.036146 (_Beijing_)
 * -33.868457, 151.205134 (_Sydney_)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

In [ ]:
def get_distance(lat1, lon1, lat2, lon2):
    return #пишем функцию, возвращающую расстояние между точками

# теперь записываем координаты офисов
office_coordinates = []

def get_min_distance_to_office(lat, lon):
    min_dist = None
    for (of_lat, of_lon) in office_coordinates:
        # здесь пишем код по отысканию минимума среди расстояний
    return min_dist

In [ ]:
cluster_df['min_distance'] = map(get_min_distance_to_office, cluster_df.cent_latitude, cluster_df.cent_longitude)

In [ ]:
cluster_df.sort_values('min_distance')[:20]

In [1]:
# в качетсве ответа прикрепляем список из 20 локаций и скриншот карты с нанесёнными точками